# **Programming Assessment \#4**

Names: ABERIN, Shawn,   LIM, Kyle,  SINGSON, Raymond

More information on the assessment is found in our Canvas course.

# **Load Data**

*While you don't have to separate your code into blocks, it might be easier if you separated loading your data from actually implementation of your code. Consider placing all loading of data into the code block below.*

In [4]:
# The code is defining a function called `make_p` that takes a DataFrame `df` as input.
# lang must contain a column 'count' that contains number types
#load error data

import pandas as pd
def make_p(pd):
    total = pd['count'].sum()
    inv = 1/total
    pd['P'] = pd['count']*inv
    return pd
x=''

with open("count_1edit.txt", "r") as f:
    x = f.read().splitlines()
gen = (dat.split('\t') for dat in x)
data = [(rec[0],int(rec[1]))for rec in gen]
err = pd.DataFrame.from_records(data, columns=['error', 'count'])
err = make_p(err)

print(err)


      error  count         P
0       e|i    917  0.023469
1       a|e    856  0.021908
2       i|e    771  0.019732
3       e|a    749  0.019169
4       a|i    559  0.014307
...     ...    ...       ...
1582     |c      1  0.000026
1583     |a      1  0.000026
1584     |'      1  0.000026
1585    w|       1  0.000026
1586   t|t       1  0.000026

[1587 rows x 3 columns]


In [5]:
#loading corpus files
import nltk
nltk.download('gutenberg')
# nltk.corpus.gutenberg.fileids()

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\matth\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [6]:
from collections import Counter

import string
print("Extracting all documents from NLTK's Project Gutenberg Collection...")
all_words = Counter()
for filename in nltk.corpus.gutenberg.fileids():
  words = [word.lower() for word in nltk.corpus.gutenberg.words(filename) if word not in string.punctuation]
  all_words.update(words)
lang = pd.DataFrame.from_dict(all_words, orient='index').reset_index()


Extracting all documents from NLTK's Project Gutenberg Collection...


In [7]:
lang = lang.rename(columns={'index':'word', 0:'count'})
lang = make_p(lang)

print(lang)

           word  count             P
0          emma    866  3.892499e-04
1            by   8512  3.825976e-03
2          jane    303  1.361925e-04
3        austen      3  1.348441e-06
4          1816      1  4.494802e-07
...         ...    ...           ...
42307  endowing      1  4.494802e-07
42308   delving      1  4.494802e-07
42309  germinal      1  4.494802e-07
42310   blither      1  4.494802e-07
42311  ushering      1  4.494802e-07

[42312 rows x 3 columns]


# **Noisy Channel Model Implementation**

*Again, you don't have to follow this directly, but consider placing your implementation of the model in the code block below.*

In [8]:
# %pip install editdistpy
from editdistpy import damerau_osa as ld

def candidates(input, all_words):
      # 1 edit distance away from word
      if input in all_words: return [input]
      d = 1
      candidate_list = [w for w in all_words if ld.distance(input,w,d) > -1]
      return candidate_list

#are we still using this?
def spell_correct(candidate_list):
      n=2
      
      probable = df.loc[candidate_list].sort_values(['P'], ascending=False).head(n)
      return list(probable.index), probable['P']

In [38]:
#Kyle's error trace
def calculate_edit_type_and_edit(input_word, candidate_word):
    if len(input_word) == len(candidate_word):
        # Check for substitution
        diff_count = 0
        edit_type = "sub"
        edit = ""
        for i in range(len(input_word)):
            if input_word[i] != candidate_word[i]:
                diff_count += 1
                edit = input_word[i] + "|"
                edit += candidate_word[i]
        if diff_count == 1:
            return edit_type, edit

    elif len(input_word) + 1 == len(candidate_word):
        # Check for insertion
        edit_type = "ins"
        edit = ""
        i, j = 0, 0
        if candidate_word[0:-1] == input_word:
            edit +=  input_word[-1] + "|" + candidate_word[-2] + candidate_word[-1]
            return edit_type, edit
        else:
            while i < len(input_word) and j < len(candidate_word):
                if input_word[i] != candidate_word[j]:
                    edit += input_word[j-1] + "|" + input_word[j-1] + candidate_word[j]
                    j += 1
                else:
                    i += 1
                    j += 1
            if j == len(candidate_word):
                return edit_type, edit

    elif len(input_word) - 1 == len(candidate_word):
        # Check for deletion
        edit_type = "del"
        edit = ""
        i, j = 0, 0
        if input_word[0:-1] == candidate_word:
            edit += input_word[-1] + input_word[-2] + "|" + candidate_word[-1]
            return edit_type, edit
        else:
            while i < len(input_word) and j < len(candidate_word):
                if input_word[i] != candidate_word[j]:
                    edit += input_word[i-1] + input_word[i] + "|" + candidate_word[i - 1]
                    i += 1
                else:
                    i += 1
                    j += 1
        if i == len(input_word):
            return edit_type, edit

    # Check for transposition
    if len(input_word) == len(candidate_word) and input_word != candidate_word:
        for i in range(len(input_word) - 1):
            if input_word[i] == candidate_word[i + 1] and input_word[i + 1] == candidate_word[i]:
                edit_type = "trans"
                edit = candidate_word[i:i+2] + "|" + input_word[i:i+2]
                return edit_type, edit

    return None, None  # No valid edit type found



In [44]:
#main function
test_input = input("User Input: ")
test_input = test_input.lower()
c = candidates(test_input, all_words)
if c[0] == test_input:
    print("Word is spelled correctly")
else:
    df = pd.DataFrame(columns=['Word', 'candidate', 'edit_type', 'edit', 'P(c)', 'P(w|c)', 'P(c) x P(w|c)'])
    #show all candidates
    i = 0
    for candidate in c:
        #get probability of word
        wordP = lang.loc[lang['word'] == candidate]
        wordP = wordP.iloc[0]['P']
        edit_type, edit = calculate_edit_type_and_edit(test_input, candidate)
        #get probability of error
        try:
            editP = err.loc[err['error'] == edit]
            editP = editP.iloc[0]['P']
        except:
            editP = 0
        temp = pd.DataFrame({'Word' : test_input, 'candidate' : candidate, 'edit_type' : edit_type, 'edit' : edit, 'P(c)' : wordP, 'P(w|c)' : editP, 'P(c) x P(w|c)' : (wordP * editP)}
                            , index=[i])
        df = pd.concat([df, temp])
        i += 1

df.sort_values('P(c) x P(w|c)')
print(df)



    Word candidate edit_type  edit          P(c)    P(w|c)  P(c) x P(w|c)
0  theee     these       sub   e|s  1.281019e-03  0.000589   7.540611e-07
1  theee     there       sub   e|r  2.867684e-03  0.000307   8.807157e-07
2  theee     three       sub   e|r  5.519617e-04  0.000307   1.695171e-07
3  theee     theme       sub   e|m  8.090644e-06  0.000102   8.282593e-10
4  theee      thee       del  ee|e  2.160651e-03  0.000000   0.000000e+00
5  theee    theefe       ins  e|ef  4.494802e-07  0.000461   2.070648e-10
